In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result
from neo4j_viz.neo4j import from_neo4j
import networkx as nx 

## Setup

In [ ]:
load_dotenv('.env', override=True)
HOST = os.getenv('OSM_URL')
USERNAME = os.getenv('OSM_USER')
PASSWORD = os.getenv('OSM_PASSWORD')
DATABASE = os.getenv('OSM_DB_NAME')

In [ ]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)
driver.verify_connectivity(database=DATABASE)

In [ ]:
schema_graph = driver.execute_query(
        ''' 
        call db.schema.visualization()
        ''',
        database_=DATABASE,
        routing_=RoutingControl.READ,
        result_transformer_=Result.graph,
    )
VG = from_neo4j(schema_graph)
VG.render()

## Generate supply and demand
Skip this step if demand and supply nodes are already in the graph (or adjust it accordingly)

In [ ]:
driver.execute_query(
    ''' 
    match (n:Location)
    return min(n.point.latitude) as min_latitude,
           max(n.point.latitude) as max_latitude,
           min(n.point.longitude) as min_longitude,
           max(n.point.longitude) as max_longitude  
    ''',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
# Pick a center point of interest
center = 2014176513

In [ ]:
# Add demand and car
driver.execute_query(
    ''' 
    match (n:Location{osmid: $center})
    with n
    match (l:Location) 
    where  1_000 < point.distance(n.point, l.point) < 10_000
    with l order by rand() limit 10 
    create (d:Demand)-[:AT]->(l)
    ''',
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    center = center,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
driver.execute_query(
    ''' 
    match (n:Location{osmid: $center})
    with n
    match (l:Location) 
    where  1_000 < point.distance(n.point, l.point) < 10_000
    with l order by rand() limit 10 
    create (d:Car)-[:AT]->(l)
    ''',
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    center = center,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
# Forgot to give the Car and Locations a name :)
driver.execute_query(
    ''' 
    match (c:Car)
    with collect(c) as cars
    call (cars) {
        unwind range(0, size(cars) - 1) as i
        with i, cars[i] as car
        set car.name = "Car " + toString(i)
    }
    match (d:Demand)
    with collect(d) as demands
    call (demands) {
        unwind range(0, size(demands) - 1) as i
        with i, demands[i] as demand
        set demand.name = "Demand " + toString(i)
    }
    ''',
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    center = center,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
# Simplify visualization by adding osmid and point to Car and Demand nodes
driver.execute_query(
    ''' 
    match (n:Car|Demand)-[:AT]->(l:Location)
    set n.osmid = l.osmid,
        n.point = l.point
    ''',
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    center = center,
    result_transformer_= lambda r: r.to_df()
)

## Create some communities

In [ ]:
gds = GraphDataScience.from_neo4j_driver(driver=driver)
gds.set_database(DATABASE)
gds.version()

In [ ]:
G, res = gds.graph.project(
    "roads",        # Graph name
    ["Location"],   #  Node projection
    ["ROAD"]        #  Relationship projection
)

In [ ]:
gds.louvain.stats(G)

In [ ]:
gds.louvain.write(
    G,
    writeProperty="louvain",
    concurrency=16,
    maxIterations=10
)

In [ ]:
gds.labelPropagation.write(
    G,
    writeProperty="label_propagation",
    concurrency=16,
)

In [ ]:
G.drop()

## Some pathfinding

In [ ]:
G, res = gds.graph.cypher.project(
   '''//cypher
    MATCH (source:Location)-[r:ROAD]->(target:Location)
    RETURN gds.graph.project('myGraph', source, target,
        {
            sourceNodeProperties: { latitude: source.point.latitude, longitude: source.point.longitude },
            targetNodeProperties: { latitude: target.point.latitude, longitude: target.point.longitude },
            relationshipProperties: r { .distance, .travelTime }
        }
    )
    ''',
    database=DATABASE
)

In [ ]:
driver.execute_query(
    ''' 
    match (:Car{name:$car})-[:AT]->(source:Location)
    with source limit 1
    match (:Demand)-[:AT]->(target:Location)
    with source, target
    
    CALL gds.shortestPath.dijkstra.stream('myGraph', {
        sourceNode: source,
        targetNodes: target,
        relationshipWeightProperty: 'travelTime'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    WITH gds.util.asNode(sourceNode) as source,
         gds.util.asNode(targetNode) as target,
         index, totalCost, nodeIds, costs, path
    RETURN
        index,
        [(source)<-[:AT]-(n)  | n.name][0] AS sourceName,
        [(target)<-[:AT]-(n)  | n.name][0] AS targetName,
        totalCost
        //[nodeId IN nodeIds | gds.util.asNode(nodeId).osmid] AS locations,
        //costs
        //nodes(path) as path
    ORDER BY index
    
    ''',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    car = 'Car 5',
    result_transformer_= lambda r: r.to_df()
).head(100)

## Compute min cost flow

In [ ]:
# Fetch all cars = suppliers
# Later we can fetch within a specific area or cluster
suppliers = driver.execute_query(
    ''' 
    match (car:Car) return car.name as car_name
    ''',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()

)
suppliers.head(10)

In [ ]:
# Fetch all demands
# Later we can fetch within a specific area or cluster
demands = driver.execute_query(
    ''' 
    match (d:Demand) return d.name as demand_name
    ''',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()

)
demands.head(10)

In [ ]:
nxG = nx.DiGraph()

# Add nodes
for i, row in suppliers.iterrows():
    car = row['car_name']
    nxG.add_node(car, demand=-1)
for i, row in demands.iterrows():
    demand = row['demand_name']
    nxG.add_node(demand, demand=1)

# Run pathfinding for each car to each demand
costs = driver.execute_query(
    ''' 
    match (c:Car)-[:AT]->(s:Location)
    where c.name in $suppliers
    with collect (s) as sources
    match (d:Demand)-[:AT]->(t:Location)
    where d.name in $demands
    with sources, collect(t) as targets
    UNWIND sources as source
    CALL gds.shortestPath.dijkstra.stream('myGraph', {
        sourceNode: source,
        targetNodes: targets,
        relationshipWeightProperty: 'travelTime'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    WITH gds.util.asNode(sourceNode) as source,
        gds.util.asNode(targetNode) as target,
        index, totalCost, nodeIds, costs, path
    RETURN
        index,
        [(source)<-[:AT]-(n)  | n.name][0] AS sourceName,
        [(target)<-[:AT]-(n)  | n.name][0] AS targetName,
        totalCost
        //[nodeId IN nodeIds | gds.util.asNode(nodeId).osmid] AS locations,
        //costs
        //nodes(path) as path
    ORDER BY index
    
    ''',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    suppliers = suppliers['car_name'],
    demands = demands['demand_name'],
    result_transformer_= lambda r: r.to_df()
)

# Add edges with costs    
for i, row in costs.iterrows():
    nxG.add_edge(row['sourceName'], row['targetName'], weight=row['totalCost'], capacity=1)

In [ ]:
#costs.head(10)

In [ ]:
# Compute the min cost flow
flow_dict = nx.min_cost_flow(nxG)

In [ ]:
flow_dict

In [ ]:
G = gds.graph.get("myGraph")
G.drop()
